# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<b>Описание данных:</b><br>
* children — количество детей в семье<br>
* days_employed — общий трудовой стаж в днях<br>
* dob_years — возраст клиента в годах<br>
* education — уровень образования клиента<br>
* education_id — идентификатор уровня образования<br>
* family_status — семейное положение<br>
* family_status_id — идентификатор семейного положения<br>
* gender — пол клиента<br>
* income_type — тип занятости<br>
* debt — имел ли задолженность по возврату кредитов<br>
* total_income — ежемесячный доход<br>
* purpose — цель получения кредита<br>

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В результате анализа исходных данных выявлены следующие проблемы:
1) Отрицательный стаж. Решение - значение по абсолюту. <br>
2) Пропуски в стаже. Решение - заполнение медианным значением. <br>
3) Данные о детях также содержат отрицательные значения и несколько слишком больших показателей. Решение - в случае отрицательного значения убираем минус, в случае "20" предполагаем, что это на самом деле "2". <br>
4) Образование нужно привести к общему регистру <br>
5) Пропуски в доходе. Решение - заполнение медианным занчением. <br>
6) Стаж и доход заполнены вещественными числами. Решение - перевести в целочисленные значения. <br>
7) Видно, что потребуется леммизация цели получения кредита. <br>

## Шаг 2. Предобработка данных

### Обработка пропусков

Возьмём модуль значения стажа и заполним пропуски медианным значением.

In [2]:
data['days_employed'] = data['days_employed'].abs() #берем модуль стажа для ухода от отрицательных значений
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()) #заполняем пропуски меданой
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

### Замена типа данных

Переведём стаж и доход в тип integer.

In [3]:
data = data.astype({"days_employed": "int64", 'total_income': 'int64'}) #переводим стаж и доход в целые числа

### Обработка дубликатов

Исправим аномалии в количестве детей и приведём данные об образовании к общему регистру.

In [4]:
data['children'] = data['children'].replace(-1, 1) #заменяем отрицательное и слишком большое количество детей на правдоподобные данные
data['children'] = data['children'].replace(20, 2)
data['education'] = data['education'].str.lower() #приводим данные об образовании к общему регистру. Для решения задачи не понадобится, но данные обработать нужно

data = data.drop_duplicates().reset_index(drop = True) # удаляем дубликат, подсчёт показал, что их 71

### Лемматизация

При анализе исходных данных чётком вырисовывались четыре категории "жилье", "автомобиль", "образование", "свадьба".<br>
Было решено объединить леммизацию и категоризацию в одну функцию. В результате оказалось, что необходимо ещё учесть "недвижимость" и решение складывалось без подсчёта лемм.

In [5]:
from pymystem3 import Mystem
m = Mystem() 
def lemma_function(purpose):
    if ("жилье" in m.lemmatize(purpose)) or ('недвижимость' in m.lemmatize(purpose)):
        return "жилье"
    elif 'автомобиль' in m.lemmatize(purpose):
        return 'автомобиль'
    elif 'образование' in m.lemmatize(purpose):
        return 'образование'
    elif 'свадьба' in m.lemmatize(purpose):
        return 'свадьба'
data['purpose_group'] = data['purpose'].apply(lemma_function)
data['purpose_group'].value_counts()

жилье          10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_group, dtype: int64

### Категоризация данных

Остальная категоризация выполнена исходя из постановки задач. <br>
По количеству детей клиентов решено было разделить на три категории: "без детей", "малодетные(1-2)", "многодетные(3+)". <br>
По семейному положению - на две: "в отношениях" и "одинок / одинока".<br>
По доходу - на четыре: "до 50 тысяч", "до 100 тысяч", "до 250 тысяч", "больше 250 тысяч". За 50 тысяч я принял среднюю зарплату по России, за 100 тысяч - комфортный доход по мнению россиян, 250 тысяч - произвольная планка, нечто среднее между средним классом и людьми с большим доходом (250+)

In [6]:
def children_function(children):
    if children == 0:
        return "без детей"
    elif (children == 1) or (children == 2):
        return "малодетная семья"
    else:
        return 'многодетная семья'
data['children_group'] = data['children'].apply(children_function)

def relationship_function(family_status):
    if (family_status == 'женат / замужем') or (family_status == "гражданский брак"):
        return "в отношениях"
    else:
        return "одинок / одинока"
data['relationship'] = data['family_status'].apply(relationship_function)

def income_function(income):
    if income <= 50000:
        return "До 50 тысяч рублей"
    if 50000 < income <= 100000:
        return "До 100 тысяч рублей"
    if 100000 < income <= 250000:
        return "До 250 тысяч рублей"
    if income > 250001:
        return 'Больше 250 тысяч рублей'
data['income_group'] = data['total_income'].apply(income_function)

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
data_children = data.pivot_table(index=['children_group', 'children'], columns='debt', values = 'education', aggfunc='count')
data_children['%'] = data_children[1] / (data_children[0] + data_children[1]) * 100
data_children.head(10)

debt                              0       1         %
children_group    children                           
без детей         0         13028.0  1063.0  7.543822
малодетная семья  1          4410.0   445.0  9.165808
                  2          1926.0   202.0  9.492481
многодетная семья 3           303.0    27.0  8.181818
                  4            37.0     4.0  9.756098
                  5             9.0     NaN       NaN

**Вывод**

В результате мы видим, что клиенты без детей чаще выплачивают свои кредиты. <br>
А вот количество детей в семье на факт выплаты влияет в рамках статистической погрешности.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
data_relationship = data.pivot_table(index=['relationship', 'family_status'], columns='debt', values = 'education', aggfunc='count')
data_relationship['%'] = data_relationship[1] / (data_relationship[0] + data_relationship[1]) * 100
data_relationship.head(10)

debt                                        0    1         %
relationship     family_status                              
в отношениях     гражданский брак        3763  388  9.347145
                 женат / замужем        11408  931  7.545182
одинок / одинока Не женат / не замужем   2536  274  9.750890
                 в разводе               1110   85  7.112971
                 вдовец / вдова           896   63  6.569343

**Вывод**

Данный анализ показал, что применённая категоризация не справляется со своей задачей.<br>
Можно было оставить 5 исходных категорий и анализировать их. В результате оказалось, что вдовцы и вдовы самые стабильные плательщики.<br>
Состоящие в гражданском браке так же часто не выплачивают вовремя кредиты, как и не женатые \ не замужние.<br>
А официально зарегистрированные пары близки даже после развода сохраняют стабильность.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data_income = data.pivot_table(index=['income_group'], columns='debt', values = 'education', aggfunc='count')
data_income['%'] = data_income[1] / (data_income[1] + data_income[0]) * 100
data_income.head(10)

debt,0,1,%
income_group,,,
Больше 250 тысяч рублей,2618,194,6.899004
До 100 тысяч рублей,3760,331,8.090931
До 250 тысяч рублей,12986,1193,8.413851
До 50 тысяч рублей,349,23,6.182796


**Вывод**

Здесь мы видим, что люди с минимальным доходом чаще (но ненамного) выплачивают кредиты, чем клиенты, зарабатывающие большие деньги.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
data_purpose = data.pivot_table(index=['purpose_group'], columns='debt', values = 'education', aggfunc='count')
data_purpose['%'] = data_purpose[1] / (data_purpose[1] + data_purpose[0]) * 100
data_purpose.head(10)

debt,0,1,%
purpose_group,,,
автомобиль,3903,403,9.359034
жилье,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


**Вывод**

Клиенты, берущие кредит на автомобиль и образование, реже возвращают свои кредиты, чем те, кому нужны деньги на жилье и свадьбу.

## Шаг 4. Общий вывод

В результате проекта применены знания, полученные в результате двух предыдущих спринтов.

Произведена предобработка данных: заполнены пробелы, строковые значения приведены к общему регистру, изменены типы данных, выполнена лемматизация и категоризация.

Проанализированы зависимости между количеством детей, семейным положением, доходом, целями получения кредита и возвратом кредита в срок.<br>
Анализ по семейному положению и доходу показал довольно неочевидные результаты, судя по количеству вдов и небогатых людей, исправно возвращающих кредиты. <br>
На мой взгляд прямая зависимость не отражает реальную картину, у каждого из разобранных (и не разобранных) факторов, есть свой весовой коэффициент, влияющий на итоговый результат и корректные выводы можно сделать, применив комплексный подход. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.